## Contextual Compression

### Overview
Contextual Compression is method of the to transform your retrieved documents

Contextual Compression refers to the process of extracting information from your retrieved docs that you think will be relevant to your final answer. It's all about increasing the signal-to-noise ratio.

Contextual compression works by iterating over your retrieved documents, then passing them through a LLM which will extract information according to context you specify in a prompt.

You're compressing your final docs based on context you give it, "contextual compression" get it?

The key component here is the "compressor" which will do our extraction/compressing for us.

Say you wanted to know everything about bananas, but you retrieved document talks about apples, oranges, and bananas. The compressor will pull out everything about bananas and then pass it on to your final prompt for a response.

### Why is this helpful?
The problem with vanilla retrieval is once you chunk your original documents, your chunks may have multiple semantic topics held within them. If you were to pass those multiple topics to the LLM, you may confuse it.

Contextual Compression is helpful when you want to try and refine your retrieved documents further.

### What are the downsides?
You'll be doing an addition API call(s) based on the number of retrieved documents you have. This will increase costs and latency of your application.

In [ ]:
%pip install openai python-dotenv --quiet

Let's start by grabbing our keys

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

Then let's import the packages we'll need

In [6]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.document_loaders import WebBaseLoader
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    azure_deployment="gpt-4",
    temperature=0,
)

and load up some data

In [4]:
# Loading a single website
loader = WebBaseLoader("http://www.paulgraham.com/superlinear.html")
docs = loader.load()

# Split your website into big chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7000, chunk_overlap=0)
chunks = text_splitter.split_documents(docs)

print (f"Your {len(docs)} documents have been split into {len(chunks)} chunks")

Your 1 documents have been split into 5 chunks


Then we'll get our embeddings and vectorstore ready

In [7]:
embedding = AzureOpenAIEmbeddings(
    azure_endpoint=os.getenv("AZURE_OPENAI_EMBEDDING_ENDPOINT"),
    openai_api_key=os.getenv("AZURE_OPENAI_EMBEDDING_API_KEY"),
    openai_api_version=os.getenv("AZURE_OPENAI_EMBEDDING_API_VERSION"),
    azure_deployment="text-embedding-3-small",
)
vectordb = Chroma.from_documents(documents=chunks, embedding=embedding)

We first need to set up our compressor, it's cool that it's a separate object because that means you can use it elsewhere outside this retriever as well.

In [8]:
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=vectordb.as_retriever(search_kwargs={"k": 2}))

Great, now that we have our compressor set up, let's try vanilla retrieval first to see what we have

In [10]:
relevant_docs = compression_retriever.base_retriever.invoke("What do people think is a flaw of capitalism?")
print (f"You have {len(relevant_docs)} relevant docs")
print (relevant_docs[0].page_content[:500])
print (f"Your first document has length: {len(relevant_docs[0].page_content)}")

You have 2 relevant docs
gradual improvements in technique, not the discoveries of a few
exceptionally learned people.[3]
It's not mathematically correct to describe a step function as
superlinear, but a step function starting from zero works like a
superlinear function when it describes the reward curve for effort
by a rational actor. If it starts at zero then the part before the
step is below any linearly increasing return, and the part after
the step must be above the necessary return at that point or no one
would bo
Your first document has length: 3920


Great, now let's see what this same document looks like, but compressed based on the context we give it.

We are going to pass a question to the compressor and with that question we will compress the doc. The cool part is this doc will be contextually compressed, meaning the resulting file will only have the information relevant to the question.

In [11]:
compressor.compress_documents(documents=relevant_docs, query="What do people think is a flaw of capitalism?")

[Document(page_content='Perhaps monopoly or regulation make it hard to compete.\nPerhaps customers have bad taste or have broken procedures for\ndeciding what to buy. There are huge swathes of mediocre things\nthat exist for such reasons.', metadata={'language': 'No language found.', 'source': 'http://www.paulgraham.com/superlinear.html', 'title': 'Superlinear Returns'}),
 Document(page_content="Some think this is a flaw of capitalism, and that if we changed the rules it would stop being true. But superlinear returns for performance are a feature of the world, not an artifact of rules we've invented. We see the same pattern in fame, power, military victories, knowledge, and even benefit to humanity. In all of these, the rich get richer.", metadata={'language': 'No language found.', 'source': 'http://www.paulgraham.com/superlinear.html', 'title': 'Superlinear Returns'})]

Great so we had a long document, now we have a shorter document with more dense information. Great for getting rid of the fluff. Let's try it out on our essays

In [12]:
question = "What do people think is a flaw of capitalism?"
compressed_docs = compression_retriever.get_relevant_documents(question)

We now have docs but they are shorter and only contain the information that is relevant to our query.

Let's put it in our prompt template again.

In [13]:
prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [19]:
output_parser = StrOutputParser()

chain = PROMPT | llm | output_parser

chain.invoke({
    "context": compressed_docs,
    "question": question
})

'Some people think that the phenomenon of superlinear returns, where the rich get richer, is a flaw of capitalism.'